### Initating Spark Session

In [ ]:
# The code was removed by Watson Studio for sharing.

### Reading data

In [ ]:
readcsv = lambda filename: spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url(filename, 'movielens-donotdelete-pr-zexbejr3wdxjtx'), inferSchema='true')

In [ ]:
genome_scores = readcsv("genome-scores.csv")
genome_scores.cache()
genome_scores.show(5)

In [ ]:
genome_tags = readcsv("genome-tags.csv")
genome_tags.show(5)

In [ ]:
links = readcsv("links.csv")
links.show(5)

In [ ]:
movies = readcsv("movies.csv")
movies.cache()
movies.show(5)

In [ ]:
ratings = readcsv("ratings.csv")
ratings.show(5)

In [ ]:
tags = readcsv("tags.csv")
tags.show(5)

## 1) Users' tag assignment reliability
<img src="https://camo.githubusercontent.com/510767150d442634b53bfb2d0f63c4739b342e20/687474703a2f2f7777772e696e646961656475636174696f6e2e6e65742f696d6167657376725f63652f3639382f43726974696373253230312e6a7067">

### 1.1) Which "User" gave which "Tag" to which "Movie"?

In [ ]:
userId_tagId_movieId = tags.join(genome_tags, on="tag").select(["userId","tagId","movieId"])
userId_tagId_movieId.cache()
userId_tagId_movieId.show(10)

### 1.2) How relevant each tag assignment was

In [ ]:
userId_tagId_movieId_relevance = userId_tagId_movieId.join(genome_scores, on=["tagId","movieId"])
userId_tagId_movieId_relevance.show(10)

### 1.3) Calculating user reliability (ranges btw 0 --> 1)
#### Irrelevant tag assignments are punished
First multiply relevance with 10 and then substract $8$ so relevance score range will be between $-6$ and $6$ and then apply sigmoid function.
Relevance scores less than or equal to $0.2$ will go closer to $0$.
- $ R = \sigma(r * 15 - 9) $
- $R$ Reliability score
- $r$ Relevance score
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/320px-Logistic-curve.svg.png">

In [ ]:
sigmoid = udf(lambda x: 1/(1+exp(-x)))

In [ ]:
users_reliability = userId_tagId_movieId_relevance.select(["userId","relevance"])\
                                            .withColumn("reliability", sigmoid(userId_tagId_movieId_relevance.relevance*15-9).cast(DoubleType()))\
                                            .groupBy("userId").avg("reliability")\
                                            .select(["userId", col("avg(reliability)").alias("reliability")])\
                                            .sort(desc("reliability"))

users_reliability.show(10)

## 2) Most reliably rated movies
Ratings are multiplied by users' reliability scores

In [ ]:
userId_movieId_ratings  = ratings.join(users_reliability, on="userId")\
                                 .withColumn("rating", round(col("rating")*col("reliability"), 1))\
                                 .select(["userId", "movieId", "rating"])
userId_movieId_ratings.cache()

In [ ]:
movieId_title_rating = userId_movieId_ratings.select(["movieId", "rating"])\
                                       .groupBy("movieId").avg("rating")\
                                       .join(movies, on="movieId")\
                                       .select(["movieId", "title", col("avg(rating)").alias("rating")])\
                                       .sort(desc("rating"))

movieId_title_rating.show(10)

### Most rated movie
<img src="https://images-na.ssl-images-amazon.com/images/I/81dt7grDwyL._RI_.jpg" width=400>

## 3) Most reliably rated tags

In [ ]:
tagId_tag_rating = userId_movieId_ratings.join(userId_tagId_movieId, on=["movieId", "userId"])\
                                   .select(["tagId", "rating"])\
                                   .groupBy("tagId").avg("rating")\
                                   .join(genome_tags, on="tagId")\
                                   .select(["tagId", "tag", col("avg(rating)").alias("rating")])\
                                   .sort(desc("rating"))

tagId_tag_rating.show(10)

## 3) Most reliably rated genres

In [ ]:
genre_rating = reliable_ratings.join(movies, on="movieId")
# TODO flatten genres "Adventure|Animation|Children|Comedy|Fantasy" ---> "Adventure", "Animation", "Children", "Comedy", "Fantasy"